In [1]:
import requests
from bs4 import BeautifulSoup
import wget
import logging
from RequestInferSchemaToJsonAPI.main import TriggerInferShemaToJsonAPI
import os
import urllib.request
from urllib import request
import pdfplumber
import tabula 
import csv
import pandas as pd
from unidecode import unidecode
import pandas as pd
base_path='c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\test_demobook'
logging.basicConfig(handlers=[logging.FileHandler("out.log"),
                              logging.StreamHandler()], level=logging.INFO)
os.getcwd()

'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data'

In [2]:
result = requests.get('https://unstats.un.org/unsd/demographic-social/products/dyb/dyb_2021/')
soup = BeautifulSoup(result.content,'html.parser')

In [3]:
index=[0,6,11,15,19,23,29,32]
#pdf_index = [34,37]

In [6]:
for i in index:
    url = "https://unstats.un.org"
    parent_node = soup.find(class_ = 'sub1')
    child_node = parent_node.find_all('li')[i]
    files = child_node.find('ul',{'class':'leaf'})
    #top =child_node.find_all("li")[files.index(files)].text
    tags = child_node.text.partition('\r')[0]
    table = files.find_all('li')
    for t in table:
        title = t.text.partition('\n')[0].replace('/', ' or ').replace('\t',' ').replace(':','')
        link_xls = t.find_all('a')[1]['href']
        link_xls = str(url) + link_xls
        link_pdf = t.find_all('a')[2]['href']
        link_pdf = str(url) + link_pdf
        conten = []
        urllib.request.urlretrieve(link_xls,f"{base_path}\\{title}.xls")
        try:
            urllib.request.urlretrieve(link_pdf,f"{base_path}\\{title}.pdf")
        except:
            pass


        BodyDict = {
            "JobPath":"", #* Point to downloaded data for conversion #
            "JsonDetails":{
                    ## Required
                    "organisation": "un-agencies",
                    "source": "UNSTATS",
                    "source_description" : "The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.",
                    "source_url" : "https://unstats.un.org/unsd/demographic-social/products/dyb/dyb_2021/",
                    "table" : title,
                    "description" : "",
                    ## Optional
                    "JobType": "JSON",
                    "CleanPush": True,
                    "Server": "str",
                    "UseJsonFormatForSQL":  False,
                    "CleanReplace":True,
                    "MergeSchema": False,
                    "tags": [{
                        "name": tags
                    }],
                    "additional_data_sources": [{
                        "name": ""
                    }],
                    "limitations":"",
                    "concept":  "",
                    "periodicity":  "",
                    "topic":  "",
                    "created": "",                       #* this should follow the following formats %Y-%m-%dT%H:%M:%S" or "%Y-%m-%d"
                    "last_modified": "",                #* ""               ""                  ""              ""
                    "TriggerTalend" :  False,    #* initialise to True for production
                    "SavePathForJsonOutput": "" #* initialise as empty string for production.
                }
            }
        if title.startswith('Table 3a'):
            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            title = 'Table 3a - Whipple s Index by sex and urban or rural residence, 1985-2021'
            df.iloc[2:4] = df.iloc[2:4].fillna(method='ffill', axis=1)
            df.iloc[2:4] = df.iloc[2:4].fillna('')
            df.columns = df.iloc[2:4].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[4:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = title
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

        elif title == 'Table 1 - Population, rate of increase, birth and death rates, surface area and density for the world, major areas and regions selected years':

            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:4] = df.iloc[2:4].fillna(method='ffill', axis=1)
            df.iloc[2:4] = df.iloc[2:4].fillna('')
            df.columns = df.iloc[2:4].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[4:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('Composition of major areas and regions')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

        elif title =='Table 2 - Estimates of population and its percentage distribution, by age and sex and sex ratio for all ages for the world, major areas and regions 2021':
            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:5] = df.iloc[2:5].fillna(method='ffill', axis=1)
            df.iloc[2:5] = df.iloc[2:5].fillna('')
            df.columns = df.iloc[2:5].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[5:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            df.rename(columns={df.columns[13]: df.columns[13].replace(' - Female \n Féminin - 65+','')},inplace=True)
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of  variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of  variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('1 United Nations,')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

        elif title == 'Table 21 - Probability of dying in the five year interval following specified age (5qx), by sex latest available year, 2002 - 2021':

            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:4] = df.iloc[2:4].fillna(method='ffill', axis=1)
            df.iloc[2:4] = df.iloc[2:4].fillna('')
            df.columns = df.iloc[2:4].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[4:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])

            meta_topic = content_all.partition('\nThe life')[0].replace('\n','')
            meta_description = content_all.partition('\nThe life')[2].split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept =  content_all.partition('\nThe life')[2].split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = content_all.partition('\nThe life')[2].split('Reliability of data')[1].split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

        elif title == 'Table 3 - Population by sex, annual rate of population increase, surface area and density':
            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:4] = df.iloc[2:4].fillna(method='ffill', axis=1)
            df.iloc[2:4] = df.iloc[2:4].fillna('')
            df.columns = df.iloc[2:4].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[4:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.rename(columns={df.columns[5]: df.columns[5].replace(' - Female \n Feminin','')},inplace=True)
            df.rename(columns={df.columns[10]: df.columns[10].replace(' \n 2021','')},inplace=True)
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

        elif title == 'Table 4 - Vital statistics summary and life expectancy at birth 2017-2021':
            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:4] = df.iloc[2:4].fillna(method='ffill', axis=1)
            df.iloc[2:4] = df.iloc[2:4].fillna('')
            df.columns = df.iloc[2:4].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[4:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.rename(columns={df.columns[7]: df.columns[7].replace(' - Crude death rate \n Taux brut de mortalite','')},inplace=True)
            df.rename(columns={df.columns[13]: df.columns[13].replace(' - Femaleb \n Femininb','')},inplace=True)
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)


        elif title == 'Table 10 - Live births by age of mother and sex of child, general and age-specific fertility rates latest available year, 2012-2021':
            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:4] = df.iloc[2:4].fillna(method='ffill', axis=1)
            df.iloc[2:4] = df.iloc[2:4].fillna('')
            df.columns = df.iloc[2:4].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[4:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.rename(columns={df.columns[4]: df.columns[4].replace(' - Female \n Feminin','')},inplace=True)
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

        elif title == 'Table 11 - Live births and live birth rates by age of father latest available year, 2012-2021':
            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:4] = df.iloc[2:4].fillna(method='ffill', axis=1)
            df.iloc[2:4] = df.iloc[2:4].fillna('')
            df.columns = df.iloc[2:4].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[4:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.rename(columns={df.columns[2]: df.columns[2].replace(' - Both sexes \n Les deux sexes','')},inplace=True)
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

        elif title == 'Table 20 - Death and death rates by cause and sex 2016 - 2020':
            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:6] = df.iloc[2:6].fillna(method='ffill', axis=1)
            df.iloc[2:6] = df.iloc[2:6].fillna('')
            df.columns = df.iloc[2:6].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[7:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)


        elif title == 'Table 6 - Total and urban population by sex 2012-2021':
            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:5] = df.iloc[2:5].fillna(method='ffill', axis=1)
            df.iloc[2:5] = df.iloc[2:5].fillna('')
            df.columns = df.iloc[2:5].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[5:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            df.rename(columns={df.columns[5]: df.columns[5].replace(' - Percent P.100','')},inplace=True)
            df.rename(columns={df.columns[8]: df.columns[8].replace(' - Percent P.100',' ')},inplace=True)
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

        elif title == 'Table 8 - Population of capital cities and cities of 100 000 or more inhabitants latest available year, 2002-2021':
            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:5] = df.iloc[2:5].fillna(method='ffill', axis=1)
            df.iloc[2:5] = df.iloc[2:5].fillna('')
            df.columns = df.iloc[2:5].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[5:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.rename(columns={df.columns[4]: df.columns[4].replace('- Population',' ')},inplace=True)
            df.rename(columns={df.columns[8]: df.columns[8].replace('- Population',' ')},inplace=True)
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

        elif title == 'Table 17 - Maternal deaths and maternal mortality ratios 2011-2020':
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.columns = df.iloc[2]
            df = df[4:]
            df = df.loc[:, df.columns.notna()]
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

        else: 
            colum = []
            df = pd.read_excel(f'{base_path}\\{title}.xls')
            df.iloc[2:4] = df.iloc[2:4].fillna(method='ffill', axis=1)
            df.iloc[2:4] = df.iloc[2:4].fillna('')
            df.columns = df.iloc[2:4].apply(lambda x: ' - '.join(map(str,[y for y in x if y])), axis=0)
            df = df.iloc[4:]
            df = df.loc[:,~df.columns.duplicated()].copy()
            list = df.columns.to_list()
            for col in list:
                colum.append(unidecode(col.replace('\n',' \n ')))
            df.columns = colum
            df.iloc[:,0] = df.iloc[:,0].str.encode('ascii', 'ignore').str.decode('ascii')
            df.to_excel(f'{base_path}\\{title}.xlsx',index=False)
            with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                text = pdf.pages[0:2]
                for te in text:
                    content = te.extract_text()
                    conten.append(content)
            content_all = ' '.join([str(elem) for elem in conten])
            meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            meta_descript =content_all.split('Description of variables')[1]
            meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            try:
                BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                BodyDict["JsonDetails"]["table"] = title
                BodyDict["JsonDetails"]["description"] = meta_description
                BodyDict["JsonDetails"]["limitations"] = limitations
                BodyDict["JsonDetails"]["concept"] = concept
                BodyDict["JsonDetails"]["topic"] = meta_topic
                #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                logging.info(f"Conversion successful - {title} ")
                print(BodyDict)
            except  Exception as err:
                print(err)

            # if title.startswith('Table 3a'):
            #     title = 'Table 3a - Whipple s Index by sex and urban or rural residence, 1985-2021'

            #     try:
            #         BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
            #         BodyDict["JsonDetails"]["table"] = title
            #         BodyDict["JsonDetails"]["description"] = title
            #         #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
            #         #TriggerInferShemaToJsonAPIClass.TriggerAPI()
            #         #logging.info(f"Conversion successful - {title} ")
            #         #print(BodyDict)
            #     except  Exception as err:
            #         print(err)


            # else:
            #     with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

            #         text = pdf.pages[0:2]
            #         for te in text:
            #             content = te.extract_text()
            #             conten.append(content)
            #     content_all = ' '.join([str(elem) for elem in conten])
            #     meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
            #     meta_descript =content_all.split('Description of variables')[1]
            #     meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
            #     concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
            #     limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

            #     try:
            #         BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
            #         BodyDict["JsonDetails"]["table"] = title
            #         BodyDict["JsonDetails"]["description"] = meta_description
            #         BodyDict["JsonDetails"]["limitations"] = limitations
            #         BodyDict["JsonDetails"]["concept"] = concept
            #         BodyDict["JsonDetails"]["topic"] = meta_topic
            #         #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
            #         #TriggerInferShemaToJsonAPIClass.TriggerAPI()
            #         #logging.info(f"Conversion successful - {title} ")
            #         #print(BodyDict)
            #     except  Exception as err:
            #         print(err)

INFO:root:Conversion successful - Table 1 - Population, rate of increase, birth and death rates, surface area and density for the world, major areas and regions selected years 


{'JobPath': 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\test_demobook\\Table 1 - Population, rate of increase, birth and death rates, surface area and density for the world, major areas and regions selected years.xlsx', 'JsonDetails': {'organisation': 'un-agencies', 'source': 'UNSTATS', 'source_description': 'The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as av

INFO:root:Conversion successful - Table 2 - Estimates of population and its percentage distribution, by age and sex and sex ratio for all ages for the world, major areas and regions 2021 


{'JobPath': 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\test_demobook\\Table 2 - Estimates of population and its percentage distribution, by age and sex and sex ratio for all ages for the world, major areas and regions 2021.xlsx', 'JsonDetails': {'organisation': 'un-agencies', 'source': 'UNSTATS', 'source_description': 'The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains

INFO:root:Conversion successful - Table 3 - Population by sex, annual rate of population increase, surface area and density 


...

{'JobPath': 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\test_demobook\\Table 23 - Marriages and crude marriage rates, by urban or rural residence 2017-2021.xlsx', 'JsonDetails': {'organisation': 'un-agencies', 'source': 'UNSTATS', 'source_description': 'The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.', 'source_url': 'h

INFO:root:Conversion successful - Table 24 - Marriages by age of groom and by age of bride latest available year, 2012-2021 


{'JobPath': 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\test_demobook\\Table 24 - Marriages by age of groom and by age of bride latest available year, 2012-2021.xlsx', 'JsonDetails': {'organisation': 'un-agencies', 'source': 'UNSTATS', 'source_description': 'The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.', 'source_url

INFO:root:Conversion successful - Table 25 - Divorces and crude divorce rates, by urban or rural residence 2017-2021 


{'JobPath': 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\test_demobook\\Table 25 - Divorces and crude divorce rates, by urban or rural residence 2017-2021.xlsx', 'JsonDetails': {'organisation': 'un-agencies', 'source': 'UNSTATS', 'source_description': 'The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.', 'source_url': 'htt

In [122]:
for i in index:
    url = "https://unstats.un.org"
    parent_node = soup.find(class_ = 'sub1')
    child_node = parent_node.find_all('li')[i]
    files = child_node.find('ul',{'class':'leaf'})
    #top =child_node.find_all("li")[files.index(files)].text
    tags = child_node.text.partition('\r')[0]
    table = files.find_all('li')
    for t in table:
        if len(t.find_all('a')) > 1:
            title = t.text.partition('\n')[0].replace('/', ' or ').replace('\t',' ').replace(':','')
            link_xls = t.find_all('a')[1]['href']
            link_xls = str(url) + link_xls
            link_pdf = t.find_all('a')[2]['href']
            link_pdf = str(url) + link_pdf
            conten = []
            urllib.request.urlretrieve(link_xls,f"{base_path}\\{title}.xls")
            try:
                urllib.request.urlretrieve(link_pdf,f"{base_path}\\{title}.pdf")
            except:
                pass


            BodyDict = {
                "JobPath":"", #* Point to downloaded data for conversion #
                "JsonDetails":{
                        ## Required
                        "organisation": "un-agencies",
                        "source": "UNSTATS",
                        "source_description" : "The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.",
                        "source_url" : "https://unstats.un.org/unsd/demographic-social/products/dyb/dyb_2021/",
                        "table" : title,
                        "description" : "",
                        ## Optional
                        "JobType": "JSON",
                        "CleanPush": True,
                        "Server": "str",
                        "UseJsonFormatForSQL":  False,
                        "CleanReplace":True,
                        "MergeSchema": False,
                        "tags": [{
                            "name": tags
                        }],
                        "additional_data_sources": [{
                            "name": ""
                        }],
                        "limitations":"",
                        "concept":  "",
                        "periodicity":  "",
                        "topic":  "",
                        "created": "",                       #* this should follow the following formats %Y-%m-%dT%H:%M:%S" or "%Y-%m-%d"
                        "last_modified": "",                #* ""               ""                  ""              ""
                        "TriggerTalend" :  False,    #* initialise to True for production
                        "SavePathForJsonOutput": "" #* initialise as empty string for production.
                    }
                }

            if title == 'Table 1 - Population, rate of increase, birth and death rates, surface area and density for the world, major areas and regions selected years':
                with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                    text = pdf.pages[0:2]
                    for te in text:
                        content = te.extract_text()
                        conten.append(content)
                content_all = ' '.join([str(elem) for elem in conten])
                meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
                meta_descript =content_all.split('Description of variables')[1]
                meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
                concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
                limitations = meta_descript.split('Limitations')[1].split('Composition of major areas and regions')[0].replace('\n','').replace('\n \n','').replace(':','')

                try:
                    BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                    BodyDict["JsonDetails"]["table"] = title
                    BodyDict["JsonDetails"]["description"] = meta_description
                    BodyDict["JsonDetails"]["limitations"] = limitations
                    BodyDict["JsonDetails"]["concept"] = concept
                    BodyDict["JsonDetails"]["topic"] = meta_topic
                    #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                    #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                    #logging.info(f"Conversion successful - {title} ")
                    #print(BodyDict)
                except  Exception as err:
                    print(err)

            elif title =='Table 2 - Estimates of population and its percentage distribution, by age and sex and sex ratio for all ages for the world, major areas and regions 2021':
                with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                    text = pdf.pages[0:2]
                    for te in text:
                        content = te.extract_text()
                        conten.append(content)
                content_all = ' '.join([str(elem) for elem in conten])
                meta_topic = content_all.split('Description of  variables')[0].replace('\n \n \n',',').replace('\n','')
                meta_descript =content_all.split('Description of  variables')[1]
                meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
                concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
                limitations = meta_descript.split('Limitations')[1].split('1 United Nations,')[0].replace('\n','').replace('\n \n','').replace(':','')

                try:
                    BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                    BodyDict["JsonDetails"]["table"] = title
                    BodyDict["JsonDetails"]["description"] = meta_description
                    BodyDict["JsonDetails"]["limitations"] = limitations
                    BodyDict["JsonDetails"]["concept"] = concept
                    BodyDict["JsonDetails"]["topic"] = meta_topic
                    #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                    #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                    #logging.info(f"Conversion successful - {title} ")
                    #print(BodyDict)
                except  Exception as err:
                    print(err)

            elif title == 'Table 21 - Probability of dying in the five year interval following specified age (5qx), by sex latest available year, 2002 - 2021':
                with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                    text = pdf.pages[0:2]
                    for te in text:
                        content = te.extract_text()
                        conten.append(content)
                content_all = ' '.join([str(elem) for elem in conten])

                meta_topic = content_all.partition('\nThe life')[0].replace('\n','')
                meta_description = content_all.partition('\nThe life')[2].split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
                concept =  content_all.partition('\nThe life')[2].split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
                limitations = content_all.partition('\nThe life')[2].split('Reliability of data')[1].split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

                try:
                    BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                    BodyDict["JsonDetails"]["table"] = title
                    BodyDict["JsonDetails"]["description"] = meta_description
                    BodyDict["JsonDetails"]["limitations"] = limitations
                    BodyDict["JsonDetails"]["concept"] = concept
                    BodyDict["JsonDetails"]["topic"] = meta_topic
                    #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                    #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                    #logging.info(f"Conversion successful - {title} ")
                    #print(BodyDict)
                except  Exception as err:
                    print(err)

            else: 
                if title.startswith('Table 3a'):
                    print('Table 3a no meta data')

                    try:
                        BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                        BodyDict["JsonDetails"]["table"] = title
                        BodyDict["JsonDetails"]["description"] = title
                        #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                        #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                        #logging.info(f"Conversion successful - {title} ")
                        #print(BodyDict)
                    except  Exception as err:
                        print(err)


                else:
                    with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                        text = pdf.pages[0:2]
                        for te in text:
                            content = te.extract_text()
                            conten.append(content)
                    content_all = ' '.join([str(elem) for elem in conten])
                    meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
                    meta_descript =content_all.split('Description of variables')[1]
                    meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
                    concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
                    limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')

                    try:
                        BodyDict["JobPath"] = f"{base_path}\\{title}.xlsx"
                        BodyDict["JsonDetails"]["table"] = title
                        BodyDict["JsonDetails"]["description"] = meta_description
                        BodyDict["JsonDetails"]["limitations"] = limitations
                        BodyDict["JsonDetails"]["concept"] = concept
                        BodyDict["JsonDetails"]["topic"] = meta_topic
                        #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                        #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                        #logging.info(f"Conversion successful - {title} ")
                        #print(BodyDict)
                    except  Exception as err:
                        print(err)


        elif len(t.find_all('a')) == 1:
            title = t.text.partition('\n')[0].replace('/', ' or ').replace('\t',' ').replace('\r','').replace(':',' ')
            link_pdf = t.find('a')['href']
            link_pdf = str(url) + link_pdf
            urllib.request.urlretrieve(link_pdf,f"{base_path}\\{title.replace(':',' ')}.pdf")
            BodyDict = {
                "JobPath":"", #* Point to downloaded data for conversion #
                "JsonDetails":{
                        ## Required
                        "organisation": "un-agencies",
                        "source": "UNSTATS",
                        "source_description" : "The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.",
                        "source_url" : "https://unstats.un.org/unsd/demographic-social/products/dyb/dyb_2021/",
                        "table" : title,
                        "description" : "",
                        ## Optional
                        "JobType": "JSON",
                        "CleanPush": True,
                        "Server": "str",
                        "UseJsonFormatForSQL":  False,
                        "CleanReplace":True,
                        "MergeSchema": False,
                        "tags": [{
                            "name": ''
                        }],
                        "additional_data_sources": [{
                            "name": ""
                        }],
                        "limitations":"",
                        "concept":  "",
                        "periodicity":  "",
                        "topic":  tags,
                        "created": "",                       #* this should follow the following formats %Y-%m-%dT%H:%M:%S" or "%Y-%m-%d"
                        "last_modified": "",                #* ""               ""                  ""              ""
                        "TriggerTalend" :  False,    #* initialise to True for production
                        "SavePathForJsonOutput": "" #* initialise as empty string for production.
                    }
                }
            if title == 'I - Annual mid-year population, United Nations estimates  2012 - 2021':
                with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                    text_source = pdf.pages[-1]
                    content_pdf = text_source.extract_text()

                source = content_pdf.split('-OrganisationdesNations\nUnies,')[0].replace('\n',' ')
                print(source)
                dfs_1= tabula.read_pdf(f"{base_path}\\{title}.pdf", pages='all', encoding='latin-1')

                with open(f'{base_path}\\{title}.csv','w') as f:
                    headerList =['Continent and country or area','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
                    dw = csv.DictWriter(f, delimiter=',',
                                        fieldnames=headerList)
                    dw.writeheader()
                    writer = csv.writer(f) 
                    writer.writerow(['Population estimates in thousands'])
                    for df in dfs_1:
                        df.to_csv(f,index=False)
                        f.write("\n")

                try:
                    BodyDict["JobPath"] = f"{base_path}\\{title}.csv"
                    BodyDict["JsonDetails"]["table"] = title
                    BodyDict["JsonDetails"]["description"] = title
                    BodyDict["JsonDetails"]['additional_data_sources'] = source
                    #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                    #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                    logging.info(f"Conversion successful - {title} ")
                    print(BodyDict)
                except  Exception as err:
                    print(err)
            elif title == 'II -  Vital statistics summary, United Nations medium variant projections  2020 - 2025':
                with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

                    text_source = pdf.pages[-1]
                    content_pdf = text_source.extract_text()
                source = content_pdf.split('27.8 5.0 19.1 69.4 72.8 3.62 22.8\n')[1].split('OrganisationdesNations\nUnies,')[0].replace('\n',' ')

                dfs_2 = tabula.read_pdf(f"{base_path}\\{title}.pdf", pages='all', encoding='latin-1')

                with open(f'{base_path}\\{title}.csv','w',encoding="utf-8") as f:
                    headerList_2 =['Continent and country or area','Crude birth rate - Taux','Crude death rate','Infant mortality rate','Male - Life expectancy at birth','Female - Life expectancy at birth','Total fertility rate','Natural increase']
                    dw_2 = csv.DictWriter(f, delimiter=',',
                                        fieldnames=headerList_2)
                    dw_2.writeheader()
                    for df in dfs_2:
                        df.to_csv(f,index=False)
                        f.write("\n")
                try:
                    BodyDict["JobPath"] = f"{base_path}\\{title}.csv"
                    BodyDict["JsonDetails"]["table"] = title
                    BodyDict["JsonDetails"]["description"] = title
                    BodyDict["JsonDetails"]['additional_data_sources'] = source
                    #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
                    #TriggerInferShemaToJsonAPIClass.TriggerAPI()
                    logging.info(f"Conversion successful - {title} ")
                    print(BodyDict)
                except  Exception as err:
                    print(err)

Table 3a no meta data
SOURCE UnitedNations,DepartmentofEconomicandSocialAffairs,PopulationDivision (2019).2019RevisionofWorldPopulationProspects


Dec 27, 2022 3:30:33 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:30:34 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:30:34 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:30:34 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:30:35 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace

INFO:root:Conversion successful - I - Annual mid-year population, United Nations estimates  2012 - 2021 


{'JobPath': 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\UNStat_demogrphic_yearbook\\I - Annual mid-year population, United Nations estimates  2012 - 2021.csv', 'JsonDetails': {'organisation': 'un-agencies', 'source': 'UNSTATS', 'source_description': 'The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.', 'source_url': 'http

Dec 27, 2022 3:30:37 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:30:37 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:30:38 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:30:38 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace

INFO:root:Conversion successful - II -  Vital statistics summary, United Nations medium variant projections  2020 - 2025 


{'JobPath': 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\UNStat_demogrphic_yearbook\\II -  Vital statistics summary, United Nations medium variant projections  2020 - 2025.csv', 'JsonDetails': {'organisation': 'un-agencies', 'source': 'UNSTATS', 'source_description': 'The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.', 's

In [121]:
url = "https://unstats.un.org"
parent_node = soup.find(class_ = 'sub1')
child_node = parent_node.find_all('li')[34]
files = child_node.find('ul',{'class':'leaf'})
tags = child_node.text.partition('\r')[0]
table = files.find_all('li')
for t in table:
    title = t.text.partition('\n')[0].replace('/', ' or ').replace('\t',' ').replace('\r','').replace(':',' ')
    link_pdf = t.find('a')['href']
    link_pdf = str(url) + link_pdf
    #urllib.request.urlretrieve(link_pdf,f"{base_path}\\{title}.pdf")
    BodyDict = {
        "JobPath":"", #* Point to downloaded data for conversion #
        "JsonDetails":{
                ## Required
                "organisation": "un-agencies",
                "source": "UNSTATS",
                "source_description" : "The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.",
                "source_url" : "https://unstats.un.org/unsd/demographic-social/products/dyb/dyb_2021/",
                "table" : title,
                "description" : "",
                ## Optional
                "JobType": "JSON",
                "CleanPush": True,
                "Server": "str",
                "UseJsonFormatForSQL":  False,
                "CleanReplace":True,
                "MergeSchema": False,
                "tags": [{
                    "name": ''
                }],
                "additional_data_sources": [{
                    "name": ""
                }],
                "limitations":"",
                "concept":  "",
                "periodicity":  "",
                "topic":  tags,
                "created": "",                       #* this should follow the following formats %Y-%m-%dT%H:%M:%S" or "%Y-%m-%d"
                "last_modified": "",                #* ""               ""                  ""              ""
                "TriggerTalend" :  False,    #* initialise to True for production
                "SavePathForJsonOutput": "" #* initialise as empty string for production.
            }
        }
    #print(BodyDict)

    if title == 'I - Annual mid-year population, United Nations estimates  2012 - 2021':
        with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

            text_source = pdf.pages[-1]
            content_pdf = text_source.extract_text()

        source = content_pdf.split('-OrganisationdesNations\nUnies,')[0].replace('\n',' ')
        print(source)
        dfs_1= tabula.read_pdf(f"{base_path}\\{title}.pdf", pages='all', encoding='latin-1')

        with open(f'{base_path}\\{title}.csv','w') as f:
            headerList =['Continent and country or area','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
            dw = csv.DictWriter(f, delimiter=',',
                                fieldnames=headerList)
            dw.writeheader()
            writer = csv.writer(f) 
            writer.writerow(['Population estimates in thousands'])
            for df in dfs_1:
                df.to_csv(f,index=False)
                f.write("\n")

        try:
            BodyDict["JobPath"] = f"{base_path}\\{title}.csv"
            BodyDict["JsonDetails"]["table"] = title
            BodyDict["JsonDetails"]["description"] = title
            BodyDict["JsonDetails"]["additional_data_sources"]= source
            #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
            #TriggerInferShemaToJsonAPIClass.TriggerAPI()
            logging.info(f"Conversion successful - {title} ")
            print(BodyDict)
        except  Exception as err:
            print(err)
    elif title == 'II -  Vital statistics summary, United Nations medium variant projections  2020 - 2025':
        with pdfplumber.open(f"{base_path}\\{title}.pdf") as pdf:

            text_source = pdf.pages[-1]
            content_pdf = text_source.extract_text()
        source_2 = content_pdf.split('27.8 5.0 19.1 69.4 72.8 3.62 22.8\n')[1].split('OrganisationdesNations\nUnies,')[0].replace('\n',' ')

        dfs_2 = tabula.read_pdf(f"{base_path}\\{title}.pdf", pages='all', encoding='latin-1')

        with open(f'{base_path}\\{title}.csv','w',encoding="utf-8") as f:
            headerList_2 =['Continent and country or area','Crude birth rate - Taux','Crude death rate','Infant mortality rate','Male - Life expectancy at birth','Female - Life expectancy at birth','Total fertility rate','Natural increase']
            dw_2 = csv.DictWriter(f, delimiter=',',
                                fieldnames=headerList_2)
            dw_2.writeheader()
            for df in dfs_2:
                df.to_csv(f,index=False)
                f.write("\n")
        try:
            BodyDict["JobPath"] = f"{base_path}\\{title}.csv"
            BodyDict["JsonDetails"]["table"] = title
            BodyDict["JsonDetails"]["description"] = title
            #BodyDict["JsonDetails"]['additional_data_sources']['name'] = source_2
            #TriggerInferShemaToJsonAPIClass = TriggerInferShemaToJsonAPI(BodyDict=BodyDict)
            #TriggerInferShemaToJsonAPIClass.TriggerAPI()
            logging.info(f"Conversion successful - {title} ")
            print(BodyDict)
        except  Exception as err:
            print(err)

SOURCE UnitedNations,DepartmentofEconomicandSocialAffairs,PopulationDivision (2019).2019RevisionofWorldPopulationProspects


Dec 27, 2022 3:28:58 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:28:59 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:29:00 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:29:00 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:29:00 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace

INFO:root:Conversion successful - I - Annual mid-year population, United Nations estimates  2012 - 2021 


{'JobPath': 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\UNStat_demogrphic_yearbook\\I - Annual mid-year population, United Nations estimates  2012 - 2021.csv', 'JsonDetails': {'organisation': 'un-agencies', 'source': 'UNSTATS', 'source_description': 'The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.', 'source_url': 'http

Dec 27, 2022 3:29:02 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:29:02 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:29:03 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 3:29:03 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace

INFO:root:Conversion successful - II -  Vital statistics summary, United Nations medium variant projections  2020 - 2025 


{'JobPath': 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\UNStat_demogrphic_yearbook\\II -  Vital statistics summary, United Nations medium variant projections  2020 - 2025.csv', 'JsonDetails': {'organisation': 'un-agencies', 'source': 'UNSTATS', 'source_description': 'The Demographic Yearbook 2021 is the seventy-second issue in a series published by the United Nations since 1948. It contains tables on a wide range of demographic statistics, including a world summary of selected demographic statistics, statistics on the size, distribution and trends in national populations,fertility, foetal mortality, infant and maternal mortality, general mortality, nuptiality and divorce. Data are shown by urban/rural residence, as available. The volume provides Technical Notes, a synoptic table, a historical index and a listing of the issues of the Demographic Yearbook published to date. This issue of Demographic Yearbook contains data as available including reference year 2021.', 's

In [67]:

url = "https://unstats.un.org"
parent_node = soup.find(class_ = 'sub1')
child_node = parent_node.find_all('li')[0]
files = child_node.find('ul',{'class':'leaf'})
#top =child_node.find_all("li")[files.index(files)].text
top = child_node.text.partition('\r')[0]
table = files.find_all('li')
for t in table:
    title = t.text.partition('\n')[0].replace('/', ' or ')
    #print(title)
    link_xls = t.find_all('a')[1]['href']
    link_xls = str(url) + link_xls
    link_pdf = t.find_all('a')[2]['href']
    link_pdf = str(url) + link_pdf
    #urllib.request.urlretrieve(link_xls,f"{base_path}\\{title.replace(':','')}.xls")
    #pdf = requests.get(link_pdf)
    #print(pdf.content)
    try:
        urllib.request.urlretrieve(link_pdf,f"{base_path}\\{title.replace(':','')}.pdf")
    except:
        pass
    import pdfplumber
    with pdfplumber.open(f"{base_path}\\Table10.pdf") as pdf:

        text = pdf.pages[0:2] #and pdf.pages[1]
        for t in text:
            content = t.extract_text()
            conten.append(content)
    content_all = ' '.join([str(elem) for elem in conten])
    meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
    meta_descript =content_all.split('Description of variables')[1]
    meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
    concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
    if 'Earlier data' not in content_all:
        limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')
    else :
        limitations = meta_descript.split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')
    if 'Description of variables' not in content_all:
        meta_topic = content_all.partition('\nThe life')[0].replace('\n','')
        meta_description = content_all.partition('\nThe life')[2].split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
        meta_description=f'The life{meta_description}'
        concept =  content_all.partition('\nThe life')[2].split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
        limitations = content_all.partition('\nThe life')[2].split('Reliability of data')[1].split('Limitations')[1].split('Earlier data')[0].replace('\n','').replace('\n \n','').replace(':','')
    

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\UNStat_demogrphic_yearbook\\Table10.pdf'

In [267]:

colm = []
import pdfplumber
with pdfplumber.open(f"{base_path}\\Table 1 - Population, rate of increase, birth and death rates, surface area and density for the world, major areas and regions selected years.pdf") as pdf:

    text = pdf.pages[0:2] #and pdf.pages[1]
    for t in text:

        clean_text = t.filter(lambda obj: obj["object_type"] == "char" and "Bold" in obj["fontname"])

        col=clean_text.extract_text()#.split("\n")#[2:]
        colm.append(col)
        content = t.extract_text()
        conten.append(content)
content_all = ' '.join([str(elem) for elem in conten])
meta_topic = content_all.split('Description of variables')[0].replace('\n \n \n',',').replace('\n','')
meta_descript =content_all.split('Description of variables')[1]
meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
limitations = meta_descript.split('Limitations')[1].split('Composition of major areas and regions')[0].replace('\n','').replace('\n \n','').replace(':','')
limitations

' The estimated orders of magnitude of population and land area are subject to all the basic limitations set forth in connection with table 3, and to the same qualifications set forth for population and surface area statistics in sections 3 and 2.4 of the Technical Notes, respectively. Likewise, rates of population increase and population density are affected by the limitations of the original figures. However, it may be noted that, in compiling data for regional and major areas totals, errors in the components may tend to compensate each other and the resulting aggregates may be more reliable than the quality of the individual components would imply. Because of their estimated character, many of the birth and death rates shown should also be considered only as orders of magnitude, and not as measures of the true level of fertility or mortality.    In interpreting the population densities, one should consider that some of the regions include large segments of land that are uninhabitabl

In [265]:
colm

['Table 1 – Demographic Yearbook 2021 \nDescription of variables\nComputation\nReliability of data\nLimitations',
 ' \nComposition of major areas and regions \n \nAFRICA \n \nEastern Africa \nASIA \n \nEastern Asia \nEUROPE \n \nEastern Europe \nMiddle Africa \nSouth-Central Asia \nNorthern Europe           \nNorthern Africa \nSouth-Eastern Asia  \nSouthern Africa \n \nSouthern Europe \nWestern Africa \n \nWestern Asia ']

In [42]:
with pdfplumber.open(f"{base_path}\\Table 2 - Estimates of population and its percentage distribution, by age and sex and sex ratio for all ages for the world, major areas and regions 2021.pdf") as pdf:

    text = pdf.pages[0:2]
    for te in text:
        content = te.extract_text()
        conten.append(content)
content_all = ' '.join([str(elem) for elem in conten])
meta_topic = content_all.split('Description of  variables')[0].replace('\n \n \n',',').replace('\n','')
meta_descript =content_all.split('Description of  variables')[1]
meta_description = meta_descript.split('Reliability of data')[0].replace('\n','').replace('\n \n','').replace(':','')
concept = meta_descript.split('Reliability of data')[1].split('Limitations')[0].replace('\n','').replace('\n \n','').replace(':','')
limitations = meta_descript.split('Limitations')[1].split('1 United Nations,')[0].replace('\n','').replace('\n \n','').replace(':','')
limitations

' The data presented in this table are from the same series of estimates, prepared by the Population Division of the United Nations, presented in table 1. The estimated orders of magnitude of population are subject to all the basic limitations set forth for population statistics in section 3 of the Technical Notes. In brief, because they are estimates, these distributions by broad age groups and sex should be considered only as orders of magnitude. However, in compiling data for regional and major areas’ totals, errors in the components tend to compensate each other and the resulting aggregates may be somewhat more reliable than the quality of the individual components would imply.  In addition, data in this table are limited by factors affecting data by age. These factors are described in the technical notes for table 7. Because the age groups presented in this table are broad, these problems are minimized.    '

In [ ]:
# I - Annual mid-year population, United Nations estimates: 2012 - 2021
# II -  Vital statistics summary, United Nations medium variant projections: 2020 - 2025
# Historical index
# Supplemental Index of Datasets Published Online

In [59]:
PDF_path='c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\pdf files UNStat_demographicYearbook'


url = "https://unstats.un.org"
parent_node1 = soup.find(class_ = 'sub1')
child_node1 = parent_node1.find_all('li')[34]
files1 = child_node1.find('ul',{'class':'leaf'})
#top =child_node.find_all("li")[files.index(files)].text
tags1 = child_node1.text.partition('\r')[0]
table1 = files1.find_all('li')
#print(table)
for t in table1:
    title = t.text.partition('\n')[0].replace('/', ' or ').replace('\t',' ').replace('\r','')
    print(title)
    link_pdf = t.find('a')['href']
    link_pdf = str(url) + link_pdf 
    urllib.request.urlretrieve(link_pdf,f"{base_path}\\{title.replace(':',' ')}.pdf")

#extract_table



I - Annual mid-year population, United Nations estimates: 2012 - 2021
II -  Vital statistics summary, United Nations medium variant projections: 2020 - 2025


In [55]:
PDF_path='c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\pdf files UNStat_demographicYearbook'
import pdfplumber
with pdfplumber.open(f"{PDF_path}\\I - Annual mid-year population, United Nations estimates  2012 - 2021.pdf") as pdf:

    text_source = pdf.pages[-1]
    content_pdf = text_source.extract_text()

content_pdf.split('-OrganisationdesNations\nUnies,')[0].replace('\n',' ')


'SOURCE UnitedNations,DepartmentofEconomicandSocialAffairs,PopulationDivision (2019).2019RevisionofWorldPopulationProspects'

In [54]:
PDF_path='c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\pdf files UNStat_demographicYearbook'
import pdfplumber
with pdfplumber.open(f"{PDF_path}\\II -  Vital statistics summary, United Nations medium variant projections  2020 - 2025.pdf") as pdf:

    text_source = pdf.pages[-1]
    content_pdf = text_source.extract_text()
content_pdf.split('27.8 5.0 19.1 69.4 72.8 3.62 22.8\n')[1].split('OrganisationdesNations\nUnies,')[0].replace('\n',' ')

'SOURCE UnitedNations,DepartmentofEconomicandSocialAffairs,PopulationDivision (2019).2019RevisionofWorldPopulationProspects-'

In [76]:
import camelot

# PDF file to extract tables from
file = f"{PDF_path}\\I - Annual mid-year population, United Nations estimates  2012 - 2021.pdf"
tables = camelot.read_pdf(file)
print("Total tables extracted:", tables.n)

2022-12-23T11:40:31 - INFO - Processing page-1
INFO:camelot:Processing page-1


Total tables extracted: 0


In [6]:
import tabula
import csv
import pandas as pd
PDF_path='c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\pdf files UNStat_demographicYearbook'
dfs = tabula.read_pdf(f"{PDF_path}\\I - Annual mid-year population, United Nations estimates  2012 - 2021.pdf", pages='all', encoding='latin-1')

#for i in range(len(dfs)):
# for df in dfs:
#     print(type(df))
#     df.to_csv("tablefrompdf.csv")
with open('all_dfs.csv','w') as f:
    for df in dfs:
        # f.write('Population estimates (in thousands)')
        # f.write("\n")
        # f.write("\n")
        #headers =['Continent and country or area','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
        df.to_csv(f,index=False)
        f.write("\n")
ddf = pd.read_csv('all_dfs.csv')
with  open('all_dfs.csv','w') as add:
    add.write('Population estimates (in thousands)')
    add.write("\n")
    headers= ['Continent and country or area','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
    writer = csv.writer(add)
    writer.writerow(headers)

Got stderr: Dec 23, 2022 1:25:51 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 23, 2022 1:25:52 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 23, 2022 1:25:52 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 23, 2022 1:25:52 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 23, 2022 1:25:53 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 23, 2022 1:25:53 PM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace



In [1]:
import tabula 
import csv
import pandas as pd
PDF_path=PDF_path='c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\pdf files UNStat_demographicYearbook'
dfs = tabula.read_pdf(f"{PDF_path}\\II -  Vital statistics summary, United Nations medium variant projections  2020 - 2025.pdf", pages='all', encoding='latin-1')

with open(f'{PDF_path}\\all_dfs.csv','w',encoding="utf-8") as f:
    headerList =['Continent and country or area','Crude birth rate - Taux','Crude death rate','Infant mortality rate','Male - Life expectancy at birth','Female - Life expectancy at birth','Total fertility rate','Natural increase']
    dw = csv.DictWriter(f, delimiter=',',
                        fieldnames=headerList)
    dw.writeheader()
    for df in dfs:
        df.to_csv(f,index=False)
        f.write("\n")

Got stderr: Dec 27, 2022 9:03:21 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 9:03:23 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 9:03:23 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 9:03:24 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 9:03:24 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace



In [3]:
import tabula 
import csv
import pandas as pd
PDF_path= 'c:\\Users\\10235555\\Documents\\DataportalCrawlers\\UN_Data\\pdf files UNStat_demographicYearbook'
dfs = tabula.read_pdf(f"{PDF_path}\\I - Annual mid-year population, United Nations estimates  2012 - 2021.pdf", pages='all', encoding='latin-1')

#for i in range(len(dfs)):
# for df in dfs:
#     print(type(df))
#     df.to_csv("tablefrompdf.csv")
with open(f'{PDF_path}\\all_1_dfs.csv','w') as f:
    headerList =['Continent and country or area','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
    dw = csv.DictWriter(f, delimiter=',',
                        fieldnames=headerList)
    dw.writeheader()
    writer = csv.writer(f) 
    writer.writerow(['Population estimates in thousands'])
    for df in dfs:
        df.to_csv(f,index=False)
        f.write("\n")

Got stderr: Dec 27, 2022 9:06:01 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 9:06:02 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 9:06:03 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 9:06:03 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 9:06:04 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace
Dec 27, 2022 9:06:04 AM org.apache.pdfbox.pdmodel.graphics.color.PDICCBased fallbackToAlternateColorSpace

